## Schema Registry

The schema registry allows the creation and retrieval of schemas. Deleting schemas is not possible. Updates can be done by by creating a modified copy of the schema with an updated version

In [ ]:
import os
import time
import datetime
import base64
import uuid
import requests
from pprint import pprint
from dotenv import load_dotenv
from typing import Any, Dict, Optional, Union, cast

from prism_agent_client import Client
from prism_agent_client.models import CredentialSchemaResponse
from prism_agent_client.models import CredentialSchemaInput
from prism_agent_client.models import ErrorResponse
from prism_agent_client.types import Response
from prism_agent_client.api.schema_registry import create_schema, get_schema_by_id, lookup_schemas_by_query


### Utilitary Functions

In [ ]:
def print_schema(schema):
    if hasattr(schema, "id"):
        print("id:", schema.id)
    print("name:", schema.name)
    print("version:", schema.version)
    print("author:", schema.author)
    print("authored:", schema.authored)
    print("tags:", schema.tags)
    print("description:", schema.description)
    
def print_schema_page(schema_page):
    for schema in schema_page.contents:
        print_schema(schema)
        print()
        
troubleshooting_message = f'''
🚨 An issue occurred while attempting to interact with the PRISM Agent 🚨

- Check that the PRISM Agent you are trying to connect to is up and running, and that it is listening on the correct port. 
  You can try to connect to the Agent using a different tool to confirm that it is available. 
  (e.g. `curl --location '<host:port>/prism-agent/connections' --header 'apiKey: <key>'`) 
- Check if there are any network issues preventing the Notebook from connecting to the Agent. This can include firewalls, 
  proxies, and other network configurations.
- Ensure that the Agent URL is correct, and that the correct API Keys are provided in the variables.env file.
- If none of the above solutions work, check the logs of the Agent container to see if there are any more specific error 
  messages that can help diagnose the issue.'''

def preflight(url, api_key):
    try:
        endpoint = f'{url}/connections'
        headers = {'apiKey': api_key}
        response = requests.get(endpoint, headers=headers, timeout=15)
        if response.status_code == 200:
            print(f"URL ok: {url}")
        else:
            raise Exception(f"URL: {response.url} code: {response.status_code} content: {response.text}")
    except Exception as Ex:
        raise Exception(f'{troubleshooting_message}\n\nURL: {url}\nAPI Key: {api_key != ""}')

### Client instances

For this example we only need one Client.

note: remember to update the file variables.env with the URLs and API keys provided to you.


#### ⚠️ NOTE:
If your host operating system is a nix-based OS that is not OSX or Windows please ensure you load the `../BetaProgram/variables_linux.env` environment variables.  
To do this uncomment the following line in the cell below: `#load_dotenv("../BetaProgram/variables-linux.env")`.  
Otherwise you will encounter issues with errors such as `ConnectionRefusedError`, `ConnectError: [Errno <n>] Name or service not known`

In [ ]:
load_dotenv("../BetaProgram/variables.env")
#load_dotenv("../BetaProgram/variables-linux.env")
issuerApiKey = os.getenv('ISSUER_APIKEY')
issuerUrl = os.getenv('ISSUER_URL')

issuer_client = Client(base_url=issuerUrl, headers={"apiKey": issuerApiKey})

%xmode Minimal

preflight(issuerUrl, issuerApiKey)

%xmode Verbose

### Create Schema

Publish the new schema with attributes to the schema registry on behalf of Cloud Agent. Schema will be signed by the keys of Cloud Agent and issued by the DID that corresponds to it

In [ ]:

data = {
    "name": f"Driving License {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    "version": f"1.0.0+{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}",
    "description": "Simple credential schema for the driving licence verifiable credential. This field is not a part of W3C specification",
    "type": "https://w3c-ccg.github.io/vc-json-schemas/schema/2.0/schema.json",
    "schema": {
        "example": {
            "$id": "driving-license-1.0",
            "$schema": "https://json-schema.org/draft/2020-12/schema",
            "description": "Driving License",
            "type": "object",
            "properties": {
                "credentialSubject": {
                    "type": "object",
                    "properties": {
                        "emailAddress": {
                            "type": "string",
                            "format": "email"
                        },
                        "givenName": {
                            "type": "string"
                        },
                        "familyName": {
                            "type": "string"
                        },
                        "dateOfIssuance": {
                            "type": "datetime"
                        },
                        "drivingLicenseID": {
                            "type": "string"
                        },
                        "drivingClass": {
                            "type": "integer"
                        },
                        "required": [
                            "emailAddress",
                            "familyName",
                            "dateOfIssuance",
                            "drivingLicenseID",
                            "drivingClass"
                        ],
                        "additionalProperties": True
                    }
                }
            }
        }
    },
    "tags": [
        "driving",
        "licence",
        "id"
    ],
    "author":"did:prism:4a5b5cf0a513e83b598bbea25cd6196746747f361a73ef77068268bc9bd732ff"
}

credential_schema_input = CredentialSchemaInput.from_dict(data)
credential_schema: Response[Union[CredentialSchemaResponse, ErrorResponse]] = create_schema.sync(client=issuer_client, json_body=credential_schema_input)
print(credential_schema)
print_schema(credential_schema)

In [ ]:
print_schema(credential_schema)

### Lookup

Lookup schemas by `author`, `name`, `tags` parameters and control the pagination by `offset` and `limit` parameters

In [ ]:
schema_page = lookup_schemas_by_query.sync(client=issuer_client, tags=["driving"], limit=1)
print_schema_page(schema_page)

### Fetch
Fetch the schema by the unique identifier. Verifiable Credential Schema in json format is returned.

In [ ]:
credential_schema = get_schema_by_id.sync(client=issuer_client, guid=credential_schema.guid)
print_schema(credential_schema)